## First definition

An `Estimator` is Tensorflow's high-level **representation of a complete model**, and it has been designed for easy scaling an asynchronous training.

## First things first

In [1]:
import tensorflow as tf
import pandas as pd

### The data set

The sample program in this notebook builds and tests a model that classifies **Iris flowers** into three different species based on the size of their sepals and petals.

In [2]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [3]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train_path, test_path

8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


('/home/nxhuy/.keras/datasets/iris_training.csv',
 '/home/nxhuy/.keras/datasets/iris_test.csv')

In [4]:
train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test  = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

train.shape, test.shape

((120, 5), (30, 5))

In [5]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [6]:
train_y = train.pop('Species')
test_y  = test.pop('Species')

train_y.shape, test_y.shape

((120,), (30,))

## Overview of programming with Estimators

An `Estimator` is any class derived from `tf.estimator.Estimator`. Tensorflow provides a collection of `tf.estimator` to implement common ML algorithms. Beyond that, we can write our own custom Estimators. We'll use **pre-made** Estimators when just getting started.

To write a Tensorflow program based on pre-made Estimators, we must perform the following tasks:
- create one or more input functions
- define model's feature columns
- instantiate an Estimator, specifying the feature columns and various hyperparameters
- call one or more methods on the Estimator object, passing the appropriate input functions as the source of the data

## Create input functions

An **input function** is a function that returns a `tf.data.Dataset` object:
- `features`: a Python dictionary
    - each key is the name of feature
    - each value is an array containing all of that feature's values
- `label`: a array containing the values of the label for every example

In [7]:
# Just to demonstrate
def input_evaluation_set():
    features = {
        'SepalLength': np.array([6.4, 5.0]),
        'SepalWidth':  np.array([2.8, 2.3]),
        'PetalLength': np.array([5.6, 3.3]),
        'PetalWidth':  np.array([2.2, 1.0])
    }
    labels = np.array([2, 1])
    return features, labels

input_evaluation_set()

({'SepalLength': array([6.4, 5. ]),
  'SepalWidth': array([2.8, 2.3]),
  'PetalLength': array([5.6, 3.3]),
  'PetalWidth': array([2.2, 1. ])},
 array([2, 1]))

Our input function may generate the `features` dictionary and `label` list any way we want. But using Tensorflow's [Dataset API](https://www.tensorflow.org/guide/data) is a recommandation, which can parse all sorts of data.

In [8]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to Dataset
    dataset = tf.data.Dataset.from_tensor_slices(
        (dict(features), labels)
    )
    # Shuffle and repeat if we are in training mode
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

## Define the feature columns

A **feature column** is an object describing how the model should use the raw input data from the features dictionary. When we build an `Estimator` model, we pass it a list of feature columns that describes each of features we want the model to use. The `tf.feature_column` module provides many options for representing data to the model.

In [10]:
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

my_feature_columns

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

## Instantiate an estimator

For the Iris problem, `tf.estimator.DNNClassifier` seems like the best choice. Here's how we instantiated this Estimator:

In [11]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    hidden_units=[30, 10],
    n_classes=3
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpa5bm98jn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Train, Evaluate and Predict

### Train the model

In [15]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO

<div class="alert alert-info">
    <b>Technical note:</b> we wrap up your <code>input_fn</code> call in a <code>lambda</code> to capture the arguments while providing an input function that takes no arguments, as expected by the Estimator. 
</div>

### Evaluate the trained model

In [16]:
eval_result = classifier.evaluate(
    input_fn=lambda : input_fn(test, test_y, training=False)
)

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-07-28T21:00:59Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpa5bm98jn/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.68359s
INFO:tensorflow:Finished evaluation at 2020-07-28-21:01:00
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.53333336, averag

### Making predictions

In [17]:
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

def input_pred_fn(features, batch_size=256):
    """An input function for prediction."""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predictions = classifier.predict(
    input_fn=lambda: input_pred_fn(predict_x))

In [18]:
predictions

<generator object Estimator.predict at 0x7fd638116150>

In [19]:
for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(
        SPECIES[class_id], 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpa5bm98jn/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Setosa" (66.9%), expected "Setosa"
Prediction is "Virginica" (51.8%), expected "Versicolor"
Prediction is "Virginica" (57.0%), expected "Virginica"


## References

- https://www.tensorflow.org/tutorials/estimator/premade